In [7]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

from functools import partial

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from online_dp import SliceDPModel, MLPModule, LinearModule
from synthetics_utils import generate_multi_mode_data, generate_label_matrix

import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simulating the Simulations

In [2]:
def generate_data(N,accs,covs,op):
    #feature and label generation
    mus = [
    np.array([-3, 0]), # Mode 1: Y = -1
    np.array([3, 0]), # Mode 2: Y = 1
    np.array([6, -3]) # Mode 3: Y = -1
    ]
    props = [0.25, 0.5, 0.25]
    labels = [-1, 1, -1]
    
    X, Y, C = generate_multi_mode_data(N, mus, props, 
                                               labels, plot=False)
    
    #labeling function generation
    L, overlap_idx = generate_label_matrix(N, accs, covs, 
                                           Y, C, overlap_portion=op)
    
    return X,Y,L,overlap_idx

In [3]:
def train_models(X,L):
    m = np.shape(L)[1]
    d = X.shape[1]


    model_0 = SliceDPModel(d, LinearModule, m, accs, r=2, rw=False)
    model_0.train(X, L, batch_size=1000, n_epochs=250, lr=0.1, print_every=250)

    input_module = partial(MLPModule, middle_dims=[4,4])
    model_1 = SliceDPModel(d, input_module, m, accs, r=2, rw=False)
    model_1.train(X, L, batch_size=1000, n_epochs=250, lr=0.1, print_every=250)

    model_2 = SliceDPModel(d, LinearModule, m, accs, r=2, rw=True)
    model_2.train(X, L, batch_size=1000, n_epochs=250, lr=0.1, print_every=250)
    
    return model_0, model_1, model_2

In [4]:
def eval_model(model,X,L,Y,overlap_idx):
    overall_score = model.score(X, Y)
    slice_scores = model.score_on_LF_slices(X, Y, L)
    try:
        overlap_scores = model.score_on_LF_slices(X[overlap_idx,:], Y[overlap_idx], L[overlap_idx,:])
    except:
        overlap_scores = [0,0,0]
    return  [list(slice_scores)+list(overlap_scores)+list([overall_score])][0]

In [5]:
def plot_error(rpca_errs, amc_errs, idx, iters, vals_list):
    """
    Plot a line graph with 95% CI shaded!
    """

    #create pandas dataframe
    m_list = vals_list
    rpca_errs_dataframe = pd.DataFrame([obj[idx] for obj in rpca_errs])
    rpca_errs_dataframe['m'] = pd.Series(m_list*iters, index=rpca_errs_dataframe.index)
    amc_errs_dataframe = pd.DataFrame([obj[idx] for obj in amc_errs])
    amc_errs_dataframe['m'] = pd.Series(m_list*iters, index=amc_errs_dataframe.index)

    #create plot
    sns.lineplot(x='m', y=0, data=rpca_errs_dataframe, marker=True, palette=sns.color_palette("GnBu_d",2))
    sns.lineplot(x='m', y=0, data=amc_errs_dataframe, marker=True, palette = sns.color_palette("GnBu_d",2))
    if idx == 6:
        plt.ylabel("Accuracy (%)")
        plt.title('Overall Accuracy')
    if idx == 0:
        plt.ylabel("Accuracy (%)")
        plt.title('Head Slice Accuracy')
    if idx == 3:
        plt.ylabel("Accuracy (%)")
        plt.title('Head-Torso Overlap Accuracy')

    plt.xlabel("Overlap Portion of Head-Torso")
    plt.legend(['Normal', 'Weighted'])
    
    sns.despine()
    plt.tight_layout()

In [ ]:
N = 10000
covs = np.array([0.9, 0.9, 0.9])
m = np.shape(covs)[0]
model_0_scores = []
model_1_scores = []
model_2_scores = []

for iteration in range(50):
    for op in [0.0, 0.05, 0.1, 0.15, 0.2]: #overlap portion
        #generate data
        accs = np.array([0.75, 0.75, 0.75])
        X,Y,L,overlap_idx = generate_data(N,accs,covs,op)
       
        #train the models and score the models (I KNOW ITS UGLY)
        model_0, model_1, model_2 = train_models(X,L)
        for model, m_scores in zip([model_0, model_1, model_2],[model_0_scores, model_1_scores, model_2_scores]):
            a,b,c,d,e,f,g = eval_model(model,X,L,Y,overlap_idx)
            m_scores.append([a,b,c,d,e,f,g])
    
    np.save('op2_model_0_scores.npy', np.array(model_0_scores))
    np.save('op2_model_1_scores.npy', np.array(model_1_scores))
    np.save('op2_model_2_scores.npy', np.array(model_2_scores))

[Epoch 0] Loss: 4.160
[Epoch 249] Loss: 4.098
Finished Training
[Epoch 0] Loss: 4.285
[Epoch 249] Loss: 4.098
Finished Training
[Epoch 0] Loss: 4.311
[Epoch 249] Loss: 4.080
Finished Training
[Epoch 0] Loss: 4.268
[Epoch 249] Loss: 4.078
Finished Training
[Epoch 0] Loss: 4.225
[Epoch 249] Loss: 4.101
Finished Training
[Epoch 0] Loss: 4.213
[Epoch 249] Loss: 4.086
Finished Training
[Epoch 0] Loss: 4.245
[Epoch 249] Loss: 4.089
Finished Training
[Epoch 0] Loss: 4.226
[Epoch 249] Loss: 4.070
Finished Training
[Epoch 0] Loss: 4.198
[Epoch 249] Loss: 4.052
Finished Training
[Epoch 0] Loss: 4.170
[Epoch 249] Loss: 4.054
Finished Training
[Epoch 0] Loss: 4.164
[Epoch 249] Loss: 4.083
Finished Training
[Epoch 0] Loss: 4.197
[Epoch 249] Loss: 4.067
Finished Training
[Epoch 0] Loss: 4.186
[Epoch 249] Loss: 4.053
Finished Training
[Epoch 0] Loss: 4.184
[Epoch 249] Loss: 4.062
Finished Training
[Epoch 0] Loss: 4.182
[Epoch 249] Loss: 4.050
Finished Training
[Epoch 0] Loss: 4.183
[Epoch 249] Loss: 

[Epoch 249] Loss: 4.086
Finished Training
[Epoch 0] Loss: 4.290
[Epoch 249] Loss: 4.067
Finished Training
[Epoch 0] Loss: 4.215
[Epoch 249] Loss: 4.070
Finished Training
[Epoch 0] Loss: 4.231
[Epoch 249] Loss: 4.057
Finished Training
[Epoch 0] Loss: 4.234
[Epoch 249] Loss: 4.061
Finished Training
[Epoch 0] Loss: 4.305
[Epoch 249] Loss: 4.068
Finished Training
[Epoch 0] Loss: 4.155
[Epoch 249] Loss: 4.048
Finished Training
[Epoch 0] Loss: 4.213
[Epoch 249] Loss: 4.076
Finished Training
[Epoch 0] Loss: 4.279
[Epoch 249] Loss: 4.100
Finished Training
[Epoch 0] Loss: 4.221
[Epoch 249] Loss: 4.095
Finished Training
[Epoch 0] Loss: 4.162
[Epoch 249] Loss: 4.085
Finished Training
[Epoch 0] Loss: 4.225
[Epoch 249] Loss: 4.097
Finished Training
[Epoch 0] Loss: 4.194
[Epoch 249] Loss: 4.097
Finished Training
[Epoch 0] Loss: 4.212
[Epoch 249] Loss: 4.062
Finished Training
[Epoch 0] Loss: 4.225
[Epoch 249] Loss: 4.148
Finished Training
[Epoch 0] Loss: 4.221
[Epoch 249] Loss: 4.070
Finished Trainin

[Epoch 249] Loss: 4.064
Finished Training
[Epoch 0] Loss: 4.175
[Epoch 249] Loss: 4.058
Finished Training
[Epoch 0] Loss: 4.180
[Epoch 249] Loss: 4.074
Finished Training
[Epoch 0] Loss: 4.266
[Epoch 249] Loss: 4.070
Finished Training
[Epoch 0] Loss: 4.180
[Epoch 249] Loss: 4.062
Finished Training
[Epoch 0] Loss: 4.211
[Epoch 249] Loss: 4.142
Finished Training
[Epoch 0] Loss: 4.274
[Epoch 249] Loss: 4.063
Finished Training
[Epoch 0] Loss: 4.213
[Epoch 249] Loss: 4.071
Finished Training
[Epoch 0] Loss: 4.299
[Epoch 249] Loss: 4.072
Finished Training
[Epoch 0] Loss: 4.179
[Epoch 249] Loss: 4.071
Finished Training
[Epoch 0] Loss: 4.264
[Epoch 249] Loss: 4.058
Finished Training
[Epoch 0] Loss: 4.234
[Epoch 249] Loss: 4.059
Finished Training
[Epoch 0] Loss: 4.292
[Epoch 249] Loss: 4.044
Finished Training
[Epoch 0] Loss: 4.260
[Epoch 249] Loss: 4.104
Finished Training
[Epoch 0] Loss: 4.212
[Epoch 249] Loss: 4.162
Finished Training
[Epoch 0] Loss: 4.233
[Epoch 249] Loss: 4.077
Finished Trainin

[Epoch 249] Loss: 4.048
Finished Training
[Epoch 0] Loss: 4.269
[Epoch 249] Loss: 4.055
Finished Training
[Epoch 0] Loss: 4.187
[Epoch 249] Loss: 4.061
Finished Training
[Epoch 0] Loss: 4.245
[Epoch 249] Loss: 4.042
Finished Training
[Epoch 0] Loss: 4.297
[Epoch 249] Loss: 4.087
Finished Training
[Epoch 0] Loss: 4.243
[Epoch 249] Loss: 4.167
Finished Training
[Epoch 0] Loss: 4.342
[Epoch 249] Loss: 4.073
Finished Training
[Epoch 0] Loss: 4.254
[Epoch 249] Loss: 4.085
Finished Training
[Epoch 0] Loss: 4.224
[Epoch 249] Loss: 4.127
Finished Training
[Epoch 0] Loss: 4.327
[Epoch 249] Loss: 4.069
Finished Training
[Epoch 0] Loss: 4.279
[Epoch 249] Loss: 4.073
Finished Training
[Epoch 0] Loss: 4.209
[Epoch 249] Loss: 4.077
Finished Training
[Epoch 0] Loss: 4.183
[Epoch 249] Loss: 4.058
Finished Training
[Epoch 0] Loss: 4.169
[Epoch 249] Loss: 4.050
Finished Training
[Epoch 0] Loss: 4.206
[Epoch 249] Loss: 4.064
Finished Training
[Epoch 0] Loss: 4.221
[Epoch 249] Loss: 4.047
Finished Trainin

In [ ]:
import pandas as pd
import seaborn as sns
sns.set_context('paper')
sns.set_style('white')

plot_error(model_0_scores, model_2_scores, idx=6, iters=20, vals_list=[0.0, 0.1, 0.25])